In [0]:
# INSTALL LIBRARIES
%pip install transformers==4.36.1 datasets torch==2.0.1 soundfile

Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.


In [0]:
dbutils.library.restartPython()

In [0]:
### SPEECH TO TEXT ####
# Code to convert Speech to Text and append all the new records to a delta table
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline
from datasets import load_dataset
import os
import pandas as pd

# Load the model and processor
model_id = "openai/whisper-large-v3"
device = "cuda" if torch.cuda.is_available() else "cpu"
model = AutoModelForSpeechSeq2Seq.from_pretrained(model_id).to(device).half()
processor = AutoProcessor.from_pretrained(model_id)

s2t_reviews = pd.DataFrame(columns=['review_id','file_path','review'])

# Create the pipeline
pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    max_new_tokens=256,
    chunk_length_s=30,
    batch_size=16,
    return_timestamps=True,
    torch_dtype=torch.float16,
    device=device,
)

# Load and process the audio file - Change the path to where your audio feedbacks are stored
audio_file_dir = "../data/audio"
audio_file_list = [f"{audio_file_dir}/{file}" for file in os.listdir(audio_file_dir)]
for audio_file in audio_file_list:
    text = pipe(audio_file)['text']
    review_id = audio_file.split('/')[-1].split('.')[0]
    s2t_reviews = s2t_reviews.append({'review_id': review_id,'file_path': audio_file, 'review': text}, ignore_index=True)

2024-11-13 16:02:20.770666: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-13 16:02:21.556420: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-13 16:02:21.556469: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-13 16:02:21.559189: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-13 16:02:21.839748: I tensorflow/core/platform/cpu_feature_g

In [0]:
s2t_reviews_sdf = spark.createDataFrame(s2t_reviews)
s2t_reviews_sdf.write.mode("overwrite").saveAsTable("genai_demo.products.camera_reviews_s2t")